In [1]:
import yfinance as yf
import talib
import numpy as np
from package import indicator as ind
import pandas as pd
from pandas_datareader import data as pdr
import random
from sklearn.decomposition import PCA
import sklearn.metrics
from genetic_selection import GeneticSelectionCV
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import datetime as datetime

import warnings
from package import euklid_regressor as eur
warnings.filterwarnings("ignore")

random.seed(42)
stock = "GOLD"

In [2]:
#import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)
# download dataframe
start = datetime.datetime(2000,8,23)
end = datetime.datetime(2022,4,1)
data = pdr.get_data_yahoo("GC=F", start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [3]:
data['vwap'] = eur.vwap(data['Close'], data['Volume'])
data['SO'] = eur.Stochastic_Oscillator(data['Close'])
data["ROC_1"] =  talib.ROC(data['Close'], timeperiod=1)
data["ROC_2"] =  talib.ROC(data['Close'], timeperiod=2)
data["ROC_3"] =  talib.ROC(data['Close'], timeperiod=3)
data["ROC_4"] =  talib.ROC(data['Close'], timeperiod=4)
data["ROC_5"] =  talib.ROC(data['Close'], timeperiod=5)
data["ROC_6"] =  talib.ROC(data['Close'], timeperiod=6)
data["ROC_7"] =  talib.ROC(data['Close'], timeperiod=7)
data["ROC_30"] =  talib.ROC(data['Close'], timeperiod=30)
data["ROC_60"] =  talib.ROC(data['Close'], timeperiod=60)
data['Momentum'] = ind.momentum(data)
data['Return'] = data.Close.pct_change()
data['1 Day ROI']=ind.ROI(data,1)
data['2 Day ROI']=ind.ROI(data,2)
data['3 Day ROI']=ind.ROI(data,3)
data['4 Day ROI']=ind.ROI(data,4)
data['5 Day ROI']=ind.ROI(data,5)
data['6 Day ROI']=ind.ROI(data,6)
data['7 Day ROI']=ind.ROI(data,7)
data['10 Day ROI']=ind.ROI(data,10)
data['20 Day ROI']=ind.ROI(data,20)
data['30 Day ROI']=ind.ROI(data,30)
data['RSI_1'] = eur.rsi(data['Close'],1)
data['RSI_2'] = eur.rsi(data['Close'],2)
data['RSI_3'] = eur.rsi(data['Close'],3)
data['RSI_4'] = eur.rsi(data['Close'],4)
data['RSI_5'] = eur.rsi(data['Close'],5)
data['RSI_6'] = eur.rsi(data['Close'],6)
data['RSI_7'] = eur.rsi(data['Close'],7)
data['RSI_14'] = eur.rsi(data['Close'],14)
data['RSI_30'] = eur.rsi(data['Close'],30)
data['RSI_60'] = eur.rsi(data['Close'],60)
data['EMA_12'] = ind.EMA(data, 12)
data['EMA_26'] = ind.EMA(data, 26)
data['Up down'] = ind.clas(data, .01)
eur.macd(data,data['Close'])
#Add SRSI to our dataframe
data['SRSI_10'] = ind.SRSI(data,10)
data['SRSI_14'] = ind.SRSI(data,14)
data['SRSI_30'] = ind.SRSI(data,30)
#data['Williams_1'] = ind.Williams(data,1)
#data['Williams_2'] = ind.Williams(data,2)
#data['Williams_3'] = ind.Williams(data,3)
data['Williams_14'] = ind.Williams(data,14)
data['ATR_14'] = ind.ATR(data,14) 
data['CCI'] = ind.CCI(data,20) 
#data['SAR'] = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0)
#data['AROONOSC_7'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=7)
#data['AROONOSC_14'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=14)
#data['AROONOSC_21'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=21)
data["APO"] = talib.APO(data['Close'], fastperiod=12, slowperiod=26, matype=0)
differenced = (data['Close'] - data['Close'].shift(1))/(data['Close'].shift(1))[1:]
previous_differenced = differenced.shift(1)
data["Differenced"] = differenced
data["Previous_differenced"] = previous_differenced

data=data.drop(['Adj Close','Volume'], axis=1)
data = data.dropna()


# Cleaning

In [4]:
data = data.dropna()
data

,Open,High,Low,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,...,MACDs_12_26_9,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced
Date,,,,,,,,,,,,,,,,,,,,,
2000-11-27,270.100006,270.100006,270.100006,270.100006,264.429989,100.000000,1.426969,1.656001,1.426969,1.656001,...,-1.178658,1,1,0,-0.0,1.076851,248.028258,-0.262181,0.014270,0.002258
2000-11-28,269.799988,269.799988,269.799988,269.799988,269.779482,94.827287,-0.111077,1.314307,1.543084,1.314307,...,-0.977252,1,0.978256,0,-5.172713,0.973274,175.719452,0.149358,-0.001111,0.014270
2000-11-29,269.600006,270.100006,265.799988,266.399994,269.475322,36.206887,-1.260191,-1.369868,0.037554,0.263448,...,-0.821127,0.675563,0.715466,0.293016,-63.793113,1.41684,71.468631,0.383974,-0.012602,-0.001111
2000-11-30,266.399994,270.200012,266.200012,270.100006,269.490908,100.000000,1.388894,0.111200,0.000000,1.426969,...,-0.640666,0.792579,0.830504,1,-1.695012,1.761261,111.249115,0.850641,0.013889,-0.012602
2000-12-01,269.700012,272.000000,268.500000,268.700012,269.482902,75.862250,-0.518324,0.863370,-0.407700,-0.518324,...,-0.475286,0.704483,0.726639,0.90191,-42.856916,1.993093,124.961987,1.271156,-0.005183,0.013889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,1953.800049,1953.800049,1953.800049,1953.800049,1997.689272,34.670747,-0.397631,0.888158,1.723335,1.306651,...,20.064008,0.419588,0.425315,0.583146,-71.72325,23.806656,3.277338,5.062171,-0.003976,0.012909
2022-03-28,1932.599976,1939.599976,1932.599976,1939.599976,1999.482419,23.921258,-0.726793,-1.121533,0.154911,0.984018,...,18.708381,0.441452,0.30219,0.559679,-79.204099,23.459112,-33.902231,-1.308983,-0.007268,-0.003976
2022-03-29,1893.500000,1912.000000,1893.500000,1912.000000,1913.622200,4.439513,-1.422973,-2.139423,-2.528547,-1.270266,...,16.930409,0.450687,0.337865,0.497318,-84.188034,26.477893,-92.836214,-7.716032,-0.014230,-0.007268


# Trying to add also PCA (Feature augmentation)

In [6]:
y = data["Up down"]
normalized = ind.normalize(data.drop(columns=["Up down", "Differenced"]))
X = normalized


In [7]:
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
data["principalDf"] = principalComponents

# Save data

In [8]:
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data.to_csv("data/"+stock+".csv")

# Feature selection with GANA

In [9]:
#Remove the first 30 index which could have a value 'N'
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data=data.drop(['High','Low'], axis=1)
data

,Open,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,ROC_5,ROC_6,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-02-26,262.000000,265.799988,262.260111,100.000000,1.878112,2.823980,2.943445,3.787576,2.943445,4.194426,...,0.739942,0.617354,0.650731,-0.0,2.564901,53.497396,-2.966664,0.018781,0.009284,-6.112981
2001-02-27,267.000000,267.399994,260.678988,100.000000,0.601959,2.491376,3.442938,3.563122,4.412334,3.563122,...,0.756489,0.71914,0.700777,-0.0,2.436248,99.192155,-2.477563,0.006020,0.018781,-6.845467
2001-02-28,266.799988,266.799988,260.481623,95.121897,-0.224385,0.376223,2.261401,3.210827,3.330742,4.178048,...,0.7491,0.715798,0.700777,-4.838761,2.191416,86.734814,-1.917948,-0.002244,0.006020,-5.069529
2001-03-01,265.200012,265.200012,258.163258,82.113952,-0.599691,-0.822731,-0.225724,1.648148,2.591881,2.711077,...,0.72425,0.754017,0.652983,-17.741797,2.112557,58.200345,-1.508333,-0.005997,-0.002244,-2.453927
2001-03-02,262.100006,262.100006,257.998698,56.910626,-1.168931,-1.761612,-1.982045,-1.392017,0.459951,1.392652,...,0.767828,0.65969,0.534431,-42.741858,2.244217,9.034521,-1.273716,-0.011689,-0.005997,1.339877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,1936.599976,1936.599976,1996.996102,21.650250,0.827824,0.414809,0.435641,-0.283199,1.498951,0.420014,...,0.358739,0.497286,0.579338,-82.160185,26.095261,-21.297422,21.601914,0.008278,-0.004096,0.063167
2022-03-24,1963.599976,1961.599976,1997.968408,40.575311,1.290922,2.129433,1.711086,1.732187,1.004068,2.809223,...,0.402563,0.48823,0.59494,-66.990286,26.26923,24.477209,10.952558,0.012909,0.008278,-2.894129
2022-03-25,1953.800049,1953.800049,1997.689272,34.670747,-0.397631,0.888158,1.723335,1.306651,1.327668,0.602444,...,0.419588,0.425315,0.583146,-71.72325,23.806656,3.277338,5.062171,-0.003976,0.012909,-1.428468


In [10]:
y = data["Up down"]
y = y.astype('int')
#Normalize the data that we have filtered
normalized = ind.normalize(data.drop(columns=["Up down","Differenced"]))
X = normalized


In [11]:
random.seed(42)
np.random.seed(42)

estimator = DecisionTreeClassifier()
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=500,
        crossover_proba=0.5,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.5,
        mutation_independent_proba=0.15,
        tournament_size=4,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	500   	[  0.495808  15.436      0.056072]	[ 0.011468  8.682966  0.023035]	[ 0.466329  1.        0.001188]	[  0.536473  30.         0.084243]
1  	299   	[-19.497613  11.422     20.04833 ]	[ 446.788648    7.012269  446.763995]	[-10000.            0.            0.001188]	[     0.536473     30.        10000.      ]
2  	262   	[  0.509344  10.602      0.046102]	[ 0.011345  6.345991  0.02234 ]      	[ 0.475215  1.        0.001188]            	[  0.536474  30.         0.082644]         
3  	293   	[-39.488858   9.222     40.042451]	[ 631.221746    5.630339  631.186664]	[-10000.            0.            0.001188]	[     0.542147     24.        10000.      ]
4  	282   	[ 0.518212  8.18      0.039926]   	[ 0.01394   5.130653  0.020012]      	[ 0.47826   1.        0.000936]            	[  0.54895   24.       

In [12]:
random.seed(42)
np.random.seed(42)

estimator = linear_model.LogisticRegression(C=1e-05, penalty='none')
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=1000,
        crossover_proba=0.10,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.05,
        mutation_independent_proba=0.05,
        tournament_size=5,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	1000  	[  0.549227  15.348      0.024411]	[ 0.017547  8.652104  0.013485]	[ 0.510573  1.        0.000113]	[  0.581475  30.         0.056295]
1  	248   	[-9.431625  7.018    10.009993]   	[ 316.087595    5.124615  316.069297]	[-10000.            0.            0.000113]	[     0.581852     29.        10000.      ]
2  	276   	[-9.42386   3.99     10.003725]   	[ 316.087841    2.422375  316.069495]	[-10000.            0.            0.000113]	[     0.581852     17.        10000.      ]
3  	289   	[ 0.578228  4.196     0.004492]   	[ 0.005062  2.050264  0.004678]      	[ 0.527401  1.        0.000113]            	[  0.582042  12.         0.039357]         
4  	324   	[ 0.578888  4.678     0.005527]   	[ 0.005804  1.877849  0.004725]      	[ 0.53572   1.        0.000113]            	[  0.58261   11.       